In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import importlib as I
import data
import model

In [3]:
X, Y = data.gen_data()

In [25]:
# graph
tf.reset_default_graph()
x_ = [ tf.placeholder(shape=[None,], 
                            dtype=tf.int64, 
                            name='xi_{}'.format(t)) for t in range(20) ]
y_ = tf.placeholder(shape=[None,], dtype=tf.int64, name='y')

x_onehot = [ tf.one_hot(x_i_, 2) for x_i_ in x_ ]

init_state = tf.zeros([16, 10])

cell = tf.nn.rnn_cell.BasicRNNCell(10)
rnn_outputs, final_state = tf.nn.rnn(cell, x_onehot, init_state)

# define params
W = tf.get_variable('W', [10, 20])
b = tf.get_variable('b', [20], initializer=tf.constant_initializer(0.0))

logits = tf.matmul(rnn_outputs[-1], W) + b
#predictions = tf.nn.softmax(logits)

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_)
loss = tf.reduce_mean(losses)
train_op = tf.train.AdagradOptimizer(learning_rate=0.1).minimize(loss)


In [21]:
batchX = X[10:10+16].T
batchY = Y[10:10+16]

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_init_state = np.zeros([16, 10])
            
    feed_dict = {x_[t]: batchX[t] for t in range(20)}
    feed_dict.update({ y_ : batchY })
    feed_dict.update({init_state : train_init_state})
    
    _, lossv, predv = sess.run([train_op, loss, logits], feed_dict = feed_dict)
    

In [38]:
np.argmax(predv, axis=1)

array([ 0,  0,  0, 15,  0,  0, 15, 15,  0,  0,  0,  0,  0,  0, 15, 15])

In [46]:
I.reload(model)

<module 'model' from '/home/suriya/_/tf/TF/RNN/toys/binseq/model.py'>

In [47]:
net = model.ManyToOne(seqlen = X.shape[-1],
               num_classes = 20,
               state_size = 5,
               ckpt_path = 'ckpt/',
               batch_size = 128,
               epochs = 10000,
               learning_rate = 0.1)

<log> Building Graph...</log>


In [43]:
I.reload(data)

<module 'data' from '/home/suriya/_/tf/TF/RNN/toys/binseq/data.py'>

In [44]:
train_set = data.rand_batch_gen(X[:-10000], Y[:-10000], 128)

In [ ]:
sess, step = net.train(train_set)


>> Average train loss : 2.207269042134285

>> Average train loss : 1.3930207001566888
